In [1]:
import sys
import pandas as pd
import numpy as np
from itertools import combinations

In [2]:
atoms_pool = [12,13,22,23,24,25,26,27,28,29,30,31,40,41,42,48,49,58,62,64,74]
c = []
for i in range(5, 6):
    c += list(combinations(atoms_pool, i))
df = pd.DataFrame({'atom_list': c})
atom_dict = {12: 'Mg', 13: 'Al', 20: 'Ca', 22: 'Ti', 23: 'V', 24: 'Cr',
                  25: 'Mn', 26: 'Fe', 27: 'Co', 28: 'Ni', 29: 'Cu', 30: 'Zn',
                  31: 'Ga', 38: 'Sr', 40: 'Zr', 41: 'Nb', 42: 'Mo', 48: 'Cd',
                  49: 'In', 56: 'Ba', 58: 'Ce', 62: 'Sm', 64: 'Gd', 74: 'W'}
for an, name in atom_dict.items():
    df[name] = df.atom_list.apply(lambda x: x.count(an) / len(x))
df['Stabilized'] = -1
X = df[atom_dict.values()]
df

,atom_list,Mg,Al,Ca,Ti,V,Cr,Mn,Fe,Co,...,Nb,Mo,Cd,In,Ba,Ce,Sm,Gd,W,Stabilized
0,"(12, 13, 22, 23, 24)",0.2,0.2,0.0,0.2,0.2,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
1,"(12, 13, 22, 23, 25)",0.2,0.2,0.0,0.2,0.2,0.0,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
2,"(12, 13, 22, 23, 26)",0.2,0.2,0.0,0.2,0.2,0.0,0.0,0.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3,"(12, 13, 22, 23, 27)",0.2,0.2,0.0,0.2,0.2,0.0,0.0,0.0,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
4,"(12, 13, 22, 23, 28)",0.2,0.2,0.0,0.2,0.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20344,"(48, 49, 58, 62, 74)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.2,0.0,0.2,0.2,0.0,0.2,-1
20345,"(48, 49, 58, 64, 74)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.2,0.0,0.2,0.0,0.2,0.2,-1
20346,"(48, 49, 62, 64, 74)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.2,0.2,-1
20347,"(48, 58, 62, 64, 74)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.2,0.2,0.2,0.2,-1


# Kennard-Stone algorithm

In [3]:
def kennardstonealgorithm(x_variables, k):
    np.random.seed(0)
    x_variables = np.array(x_variables)
    original_x = x_variables
    distance_to_average = ((x_variables - np.tile(x_variables.mean(axis=0), (x_variables.shape[0], 1))) ** 2).sum(axis=1)
    max_distance_sample_number = np.where(distance_to_average == np.max(distance_to_average))
    max_distance_sample_number = max_distance_sample_number[0][0]
    selected_sample_numbers = list()
    selected_sample_numbers.append(max_distance_sample_number)
    remaining_sample_numbers = np.arange(0, x_variables.shape[0], 1)
    x_variables = np.delete(x_variables, selected_sample_numbers, 0)
    remaining_sample_numbers = np.delete(remaining_sample_numbers, selected_sample_numbers, 0)
    for iteration in range(1, k):
        sys.stdout.write('\r %i / %i' % (iteration, k))
        selected_samples = original_x[selected_sample_numbers, :]
        min_distance_to_selected_samples = list()
        for min_distance_calculation_number in range(0, x_variables.shape[0]):
            distance_to_selected_samples = ((selected_samples - np.tile(x_variables[min_distance_calculation_number, :],
                                                                        (selected_samples.shape[0], 1))) ** 2).sum(axis=1)
            min_distance_to_selected_samples.append(np.min(distance_to_selected_samples))
        max_distance_sample_number = np.where(
            min_distance_to_selected_samples == np.max(min_distance_to_selected_samples))
        max_distance_sample_number = max_distance_sample_number[0][0]
        selected_sample_numbers.append(remaining_sample_numbers[max_distance_sample_number])
        x_variables = np.delete(x_variables, max_distance_sample_number, 0)
        remaining_sample_numbers = np.delete(remaining_sample_numbers, max_distance_sample_number, 0)
 
    return selected_sample_numbers, remaining_sample_numbers

In [6]:
# n samples are selected using kennard stone algorithm
n = 500
train_idx, test_idx = kennardstonealgorithm(X, n)

 499 / 500

In [9]:
df[df.index.isin(train_idx)].to_csv('KS.txt', index=False)